In [1]:
from enum import Enum

In [2]:
class Category(Enum):
    CLOTHES = "Clothes"
    WEAPON = "Weapon"
    FOOD = "Food"
    OTHER = "Other"

In [3]:
def print_categories():
    print("Available item categories:")
    for category in Category:
        print("* " + category.value)

In [4]:
print_categories()

Available item categories:
* Clothes
* Weapon
* Food
* Other


In [5]:
class Item:

    def __init__(self, name, category, value, weight, condition=100.0):
        self.name = name
        self.category = category
        self.value = value
        self.weight = weight
        self.condition = condition

    def decrease_condition(self, amount):
        self.condition -= amount

    def repair_condition(self):
        self.condition = 100.0

    def print(self):
        print("Name: " + self.name)
        print("Category: " + self.category.value)
        print("Value: " + str(self.value))
        print("Weight: " + str(self.weight))

        if self.condition > 95.0:
            verbatim = "excellent"
        elif self.condition > 75.0:
            verbatim = "good"
        elif self.condition > 50.0:
            verbatim = "fair"
        elif self.condition > 25.0:
            verbatim = "poor"
        elif self.condition > 0.0:
            verbatim = "very poor"
        else:
            verbatim = "broken"

        print("Condition: " + verbatim + " (" + str(self.condition) + "%)")

In [6]:
# Create some items
hat = Item("Hat", Category.CLOTHES, 3.2, 0.34)
boots = Item("Boots", Category.CLOTHES, 5.6, 1.75)
sword = Item("Sword", Category.WEAPON, 13.2, 6.4)
bow = Item("Bow", Category.WEAPON, 10.7, 4.8)
bread = Item("Bread", Category.FOOD, 2.7, 0.78)
apple = Item("Apple", Category.FOOD, 0.34, 0.27)

In [7]:
print(sword.name)

Sword


In [8]:
hat.print()

Name: Hat
Category: Clothes
Value: 3.2
Weight: 0.34
Condition: excellent (100.0%)


In [9]:
hat.decrease_condition(45)
hat.print()

Name: Hat
Category: Clothes
Value: 3.2
Weight: 0.34
Condition: fair (55.0%)


In [10]:
class Gold:

    @staticmethod
    def _validate_amount_is_positive(amount):
        if amount < 0:
            raise ValueError("Amount must be positive")

    def __init__(self, amount=0.0):
        Gold._validate_amount_is_positive(amount)

        self.name = "Gold"
        self.category = Category.OTHER
        self.amount = amount

    def add_amount(self, amount):
        Gold._validate_amount_is_positive(amount)
        self.amount += amount

    def remove_amount(self, amount):
        Gold._validate_amount_is_positive(amount)

        if amount >= self.amount:
            raise ValueError("Not enough gold")

        self.amount -= amount

    def calculate_weight(self):
        return self.amount / 1000

    def print(self):
        print("Name: " + self.name)
        print("Category: " + self.category.value)
        print("Amount: " + str(self.amount))
        print("Weight: " + str(self.calculate_weight()))

In [11]:
gold = Gold(200)
gold.print()

Name: Gold
Category: Other
Amount: 200
Weight: 0.2


In [12]:
gold.add_amount(50)
gold.print()

Name: Gold
Category: Other
Amount: 250
Weight: 0.25


In [13]:
try:
    gold.add_amount(-50)
except ValueError as e:
    print(e)

Amount must be positive


In [14]:
gold.remove_amount(100)
gold.print()

Name: Gold
Category: Other
Amount: 150
Weight: 0.15


In [15]:
try:
    gold.remove_amount(500)
except ValueError as e:
    print(e)
    print()

gold.print()

Not enough gold

Name: Gold
Category: Other
Amount: 150
Weight: 0.15


In [16]:
class Inventory:

    def __init__(self, items=None, gold_amount=0.0):
        self.items = items if items is not None else []
        self.gold = Gold(gold_amount)

    def add_item(self, item):
        if item in self.items:
            raise ValueError("Item already in inventory")
        self.items.append(item)

    def remove_item(self, item):
        if item not in self.items:
            raise ValueError("Item not in inventory")
        self.items.remove(item)

    def add_gold(self, amount):
        self.gold.add_amount(amount)

    def remove_gold(self, amount):
        self.gold.remove_amount(amount)

    def print(self, category=None):
        print("Inventory")
        print("=========")
        print()
        print("Gold: " + str(self.gold.amount))
        print()
        self._print_items(category)
        print()

    def _print_items(self, category=None):
        if len(self.items) == 0:
            print("Inventory is empty")
        else:
            if category is None:
                for item in self.items:
                    item.print()
                    print()
            else:
                for item in self.items:
                    if item.category == category:
                        item.print()
                        print()

In [17]:
inventory = Inventory()
inventory.print()

Inventory

Gold: 0.0

Inventory is empty


In [18]:
inventory.add_item(hat)
inventory.print()

Inventory

Gold: 0.0

Name: Hat
Category: Clothes
Value: 3.2
Weight: 0.34
Condition: fair (55.0%)


In [19]:
try:
    inventory.add_item(hat)
except ValueError as e:
    print(e)

Item already in inventory


In [20]:
inventory.remove_item(hat)
inventory.print()

Inventory

Gold: 0.0

Inventory is empty


In [21]:
try:
    inventory.remove_item(sword)
except ValueError as e:
    print(e)

Item not in inventory


In [22]:
inventory.add_gold(250)
inventory.print()

Inventory

Gold: 250.0

Inventory is empty


In [23]:
inventory.remove_gold(100)
inventory.print()

Inventory

Gold: 150.0

Inventory is empty


In [24]:
inventory.add_item(hat)
inventory.add_item(boots)
inventory.add_item(sword)
inventory.add_item(bread)
inventory.add_item(apple)

inventory.print(Category.FOOD)

Inventory

Gold: 150.0

Name: Bread
Category: Food
Value: 2.7
Weight: 0.78
Condition: excellent (100.0%)

Name: Apple
Category: Food
Value: 0.34
Weight: 0.27
Condition: excellent (100.0%)


In [25]:
class Character:

    def __init__(self, name, health=100, inventory=None):
        self.name = name
        self.health = health
        self.inventory = inventory if inventory is not None else Inventory()

    def increase_health(self, amount):
        self.health += amount
        if self.health > 100:
            self.health = 100

    def decrease_health(self, amount):
        self.health -= amount
        if self.health < 0:
            self.health = 0

    def print(self):
        print("Name: " + self.name)

        if self.health > 95:
            verbatim = "excellent"
        elif self.health > 75:
            verbatim = "good"
        elif self.health > 50:
            verbatim = "fair"
        elif self.health > 25:
            verbatim = "poor"
        elif self.health > 0:
            verbatim = "very poor"
        else:
            verbatim = "dead"

        print("Health: " + verbatim + " (" + str(self.health) + "%)")
        print()
        self.inventory.print()

In [26]:
bob = Character("Bob")
bob.print()

Name: Bob
Health: excellent (100%)

Inventory

Gold: 0.0

Inventory is empty


In [27]:
bob.decrease_health(35)
bob.print()

Name: Bob
Health: fair (65%)

Inventory

Gold: 0.0

Inventory is empty


In [28]:
bob.inventory.add_gold(260)
bob.inventory.add_item(bow)
bob.inventory.add_item(apple)

In [29]:
bob.print()

Name: Bob
Health: fair (65%)

Inventory

Gold: 260.0

Name: Bow
Category: Weapon
Value: 10.7
Weight: 4.8
Condition: excellent (100.0%)

Name: Apple
Category: Food
Value: 0.34
Weight: 0.27
Condition: excellent (100.0%)
